# Obtain data of AGN from SIMBAD and filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED 

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

In [ ]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [1]:
from datetime import date
import time

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [2]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [3]:
today = date.today()
today

datetime.date(2021, 10, 7)

We extract the object name (or in this case the main id) from the sample result.

In [4]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]


The sample results is saved in a .csv file:

In [5]:
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because for some objects when we search the information in [NED](https://ned.ipac.caltech.edu/) using their SIMBAD main id  it doesn't reconigze it, so we will use coordinates instead to get the NED object name

In [6]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Freq** we define a list with the those frequency values

In [7]:
Ne_IR_Freq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

The next part of the code use **[Astroquery](https://astroquery.readthedocs.io/en/latest/index.html)** package to get the "Frequency" column from the photometry table for each object of the SIMBAD sample or to get the NED object name from SIMBAD object coordiantes extracted before with SkyCoord. 

So, we will try to get the **"Photometry"** table using directly the SIMBAD object name. If this isn't recognized in NED, then we try to get the NED object name from the object coordinates and after that we will try again to get the "Photometry" table but now using the NED object name.

The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the table the values of the list defined in Ne_IR_Freq.

In [10]:
Ne_IR_obj = [] #List of objects with Ne-IR data

outfile = open('Obj_with_Ne-IR.txt','w') #We create and open a file to write the list of objects names with Ne_IR data
headerstring = f'Objects with Ne II, Ne III and Ne V data (Created on {today}):\n' #Creating the header
outfile.write(headerstring) #Writing the header

start = time.time()
for ind in range(len(simbad_sample_Sy)):
    try:
        phot_t = Ned.get_table(obj_name_SMB[ind], table='photometry') #We get the Photometry Table using SIMBAD names
        Fr=phot_t['Frequency'] #We get the Frequency column from the table
        if Ne_IR_Freq[0] in Fr or Ne_IR_Freq[1] in Fr or Ne_IR_Freq[2] in Fr or Ne_IR_Freq[3] in Fr or Ne_IR_Freq[4] in Fr:
            print(f"Index #:{ind}, Name(SIMBAD): {obj_name_SMB[ind]}")
            #Writing the object names
            outstring = f'\n{obj_name_SMB[ind]}'
            outfile.write(outstring) 
            Ne_IR_obj.append(obj_name_SMB[ind]) #Save the object name in the list Ne_IR_obj
    except:
        try:
            result_t = Ned.query_region(cat_smb_sample[ind],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample 
            obj_name_NED=result_t["Object Name"] 
            phot_t = Ned.get_table(obj_name_NED, table='photometry')
            Fr=phot_t['Frequency']
            if Ne_IR_Freq[0] in Fr or Ne_IR_Freq[1] in Fr or Ne_IR_Freq[2] in Fr or Ne_IR_Freq[3] in Fr or Ne_IR_Freq[4] in Fr:
                print(f"Index #:{ind}, Name(NED): {obj_name_NED[0]}")
                #Writing the object names
                outstring = f'\n{obj_name_NED[0]}'
                outfile.write(outstring) 
                Ne_IR_obj.append(obj_name_NED[0]) #Save the object name in the list Ne_IR_obj
        except:
            pass
end = time.time()

#we close the file with objects names
outfile.close()

print('\nTotal objects with Ne II or Ne III or Ne V data found: ',len(Ne_IR_obj))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

Index #:0, Name(SIMBAD): LEDA  165630
Index #:23, Name(SIMBAD): ESO 362-18
Index #:100, Name(NED): 3C 334
Index #:182, Name(SIMBAD): ESO 323-77
Index #:342, Name(SIMBAD): NGC  7479
Index #:400, Name(SIMBAD): M  81
Index #:408, Name(SIMBAD): NGC  1241
Index #:409, Name(SIMBAD): NGC  3516
Index #:429, Name(SIMBAD): IRAS 15206+3342
Index #:450, Name(SIMBAD): LEDA 1487448
Index #:611, Name(SIMBAD): Mrk  937
Index #:624, Name(SIMBAD): ESO 297-18
Index #:644, Name(SIMBAD): IC  450
Index #:790, Name(SIMBAD): Mrk  622
Index #:858, Name(NED): WISEA J035734.06-053719.6
Index #:1051, Name(SIMBAD): 2MASS J21035875-0728025
Index #:1111, Name(SIMBAD): IRAS 15480-0344
Index #:1411, Name(SIMBAD): Mrk  471
Index #:1585, Name(SIMBAD): 2MASS J00105486+0014510
Index #:1588, Name(NED): WISEA J005439.78+155447.2
Index #:1868, Name(SIMBAD): LEDA  166692
Index #:2265, Name(SIMBAD): NGC  4992
Index #:2623, Name(SIMBAD): 3C 295
Index #:2851, Name(SIMBAD): LEDA    3237
Index #:2852, Name(SIMBAD): Mrk  478
Index 

Index #:10901, Name(SIMBAD): NGC   424
Index #:10929, Name(SIMBAD): M 106
Index #:10968, Name(SIMBAD): NGC  5252
Index #:10972, Name(SIMBAD): IC 4553
Index #:10985, Name(SIMBAD): UGC 12282
Index #:11040, Name(SIMBAD): LEDA   89032
Index #:11089, Name(SIMBAD): NGC  6240
Index #:11102, Name(SIMBAD): LEDA   90196
Index #:11107, Name(SIMBAD): NGC   262
Index #:11138, Name(SIMBAD): NGC  7679
Index #:11188, Name(SIMBAD): Mrk  352
Index #:11193, Name(SIMBAD): ESO 417-6
Index #:11194, Name(SIMBAD): NGC  1194
Index #:11197, Name(SIMBAD): Mrk    3
Index #:11210, Name(SIMBAD): IC 5063
Index #:11220, Name(SIMBAD): NGC  3976
Index #:11221, Name(SIMBAD): NGC  5273
Index #:11223, Name(SIMBAD): NGC  5899
Index #:11229, Name(SIMBAD): NGC  7592
Index #:11244, Name(SIMBAD): IC 5298
Index #:11295, Name(SIMBAD): NGC  4507
Index #:11296, Name(SIMBAD): NGC  4477
Index #:11297, Name(SIMBAD): UGC  6728
Index #:11298, Name(SIMBAD): NGC  7319
Index #:11313, Name(SIMBAD): MCG-02-08-039
Index #:11337, Name(SIMBAD)

In [62]:
Ne_IR_obj

['LEDA  165630',
 'ESO 362-18',
 '3C 334',
 'ESO 323-77',
 'NGC  7479',
 'M  81',
 'NGC  1241',
 'NGC  3516',
 'IRAS 15206+3342',
 'LEDA 1487448',
 'Mrk  937',
 'ESO 297-18',
 'IC  450',
 'Mrk  622',
 'WISEA J035734.06-053719.6',
 '2MASS J21035875-0728025',
 'IRAS 15480-0344',
 'Mrk  471',
 '2MASS J00105486+0014510',
 'WISEA J005439.78+155447.2',
 'LEDA  166692',
 'NGC  4992',
 '3C 295',
 'LEDA    3237',
 'Mrk  478',
 'NGC  1125',
 'Mrk  279',
 'Mrk 1502',
 'PB  3894',
 'ICRF J054236.1+495107',
 '3C 098',
 'NGC  2992',
 'NGC  2110',
 'UGC  3351',
 'NGC   788',
 'NGC  4235',
 'NGC  6860',
 'Mrk  520',
 'MCG+04-48-002',
 'ICRF J145907.5+714019',
 '2E  2503',
 'MCG+00-29-023',
 'Z 218-7',
 'Mrk  463E',
 'LEDA   27453',
 'IRAS 07598+6508',
 'NGC  4151',
 'NGC  1365',
 '2XMM J101756.8+390526',
 '3C  84',
 'ESO 141-55',
 'MCG+10-17-021',
 'Mrk  509',
 'NGC  7591',
 'Mrk  493',
 'Mrk 1239',
 'Mrk  231',
 'NGC  3079',
 'NGC  4355',
 'IRAS 13342+3932',
 'WISEA J165939.77+183436.7',
 'Mrk  876',

Las celdas a continuación son para pruebas:

In [72]:
result_t

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,str30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,WISEA J032518.46-060738.4,51.32687,-6.1274,QSO,170864.0,0.56994,,21.3g,0.0,23,0,42,6,11,8,1


In [73]:
simbad_sample_Sy[ind]

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
2XMMi J032518.4-060739,51.326875,-6.1273888888888886,2005AJ....129.1755A,Sy1,2009ApJS..182..543A,126754.92483174153,0.57005,z,13,QSO|X |Sy1|AGN


In [74]:
cat_smb_sample[ind]

<SkyCoord (ICRS): (ra, dec) in deg
    (51.326875, -6.12738889)>

In [25]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.27.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 21 Model 2 Stepping 0, AuthenticAMD
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.3.1

astropy: 4.3.1
numpy  : 1.20.3
pyvo   : 1.1

Watermark: 2.2.0

